In [2]:
import numpy as np
import matplotlib.pyplot as plt
from obspy.core import read, Stream
import pandas as pd
from scipy.interpolate import interp1d

In [3]:
sw4_path = '/Users/sydneydybing/SW4/strain/'
real_path = '/Users/sydneydybing/StrainProject/2024/'

In [4]:
# Event info

# Read in event location file 
event_file = 'M6_catalog_2004-2024.csv'
eventLocs = pd.read_csv(real_path + event_file)

# Pick your event
event = eventLocs.iloc[11] # Ridgecrest mainshock

hypo_lat = event.latitude
hypo_lon = event.longitude
hypo_depth = event.depth # km

In [5]:
# Stations

sta_file = 'sw4_strain_stations_metadata.csv'
stas = pd.read_csv(sw4_path + sta_file)

In [6]:
stas.BNUM.values

array(['B072', 'B079', 'B082', 'B087', 'B916', 'B917', 'B918', 'B921'], dtype=object)

In [25]:
for idx in range(len(stas)):
    
    sta = stas.BNUM.values[idx]
    theo_p_arr = stas.p_arrival.values[idx]
    hypo_dist = stas.hypo_dist_km.values[idx]
    
    rms_sw4_strain_both = np.load(sw4_path + 'final_RC_model_outputs_processed/strain/rms/' + str(sta) + '.npy')
    filter_rms_sw4_strain_both = np.load(sw4_path + 'final_RC_model_outputs_processed/strain/filtered/rms/' + str(sta) + '.npy')
    
    time_sw4_strain = rms_sw4_strain_both[:,0]
    rms_sw4_strain = rms_sw4_strain_both[:,1]
    filter_time_sw4_strain = filter_rms_sw4_strain_both[:,0]
    filter_rms_sw4_strain = filter_rms_sw4_strain_both[:,1]
    
    # Load the real noise
    if sta != 'B918':
        rms_real_noise_st = read(real_path + 'strain_data/noise/rms/2019-07-06_M7.1/' + str(sta) + '.mseed')
        filter_rms_real_noise_st = read(real_path + 'strain_data/noise/3comp_filtered_rms/2019-07-06_M7.1/' + str(sta) + '.mseed')
    else:
        rms_real_noise_st = read(real_path + 'strain_data/noise/rms/2019-08-29_M6.3/' + str(sta) + '.mseed')
        filter_rms_real_noise_st = read(real_path + 'strain_data/noise/3comp_filtered_rms/2019-08-29_M6.3/' + str(sta) + '.mseed')
    
    time_noise_strain = rms_real_noise_st[0].times()
    rms_noise_strain = rms_real_noise_st[0].data
    filter_time_noise_strain = filter_rms_real_noise_st[0].times()
    filter_rms_noise_strain = filter_rms_real_noise_st[0].data
    
    # Interpolate SW4 strain to 20 Hz
    f = interp1d(time_sw4_strain, rms_sw4_strain)
    interp_rms_sw4_strain = f(time_noise_strain)
    g = interp1d(filter_time_sw4_strain, filter_rms_sw4_strain)
    interp_filter_rms_sw4_strain = f(filter_time_noise_strain)
    
    # Add noise to SW4 strain
    noisy_sw4_rms_strain = interp_rms_sw4_strain + rms_noise_strain
    noisy_sw4_filter_rms_strain = interp_filter_rms_sw4_strain + filter_rms_noise_strain
    
    # plt.plot(time_noise_strain, interp_rms_sw4_strain, label = 'Interpolated RMS')
    # plt.plot(time_noise_strain, noisy_sw4_rms_strain, label = 'Noisy RMS')
    # plt.plot(filter_time_noise_strain, noisy_sw4_filter_rms_strain, linestyle = '--', label = 'Noisy filtered RMS')
    # plt.xlim(40,60)
    # plt.legend()
    # plt.semilogy()
    # plt.show()
    
    # Rename variables to match old processing
    time_strain = time_noise_strain
    rms_strain = interp_rms_sw4_strain
    noisy_rms_strain = noisy_sw4_rms_strain
    filter_rms_strain = interp_filter_rms_sw4_strain
    noisy_filter_rms_strain = noisy_sw4_filter_rms_strain
    
    # Calculate peak strain - noisy rms and noisy filtered rms
    strain_p_i = np.abs(time_strain - theo_p_arr).argmin() # Get the index where the P-wave arrives
    
    mod_noisy_rms_strain = noisy_rms_strain.copy()
    noisy_p_arr_strain = noisy_rms_strain[strain_p_i]
    mod_noisy_rms_strain[:strain_p_i] = noisy_p_arr_strain # Strain before the P wave is set to be the same as the strain AT the P wave
    noisy_peak_strain = mod_noisy_rms_strain.copy()
    for k in range(0,len(mod_noisy_rms_strain)):
        if k == 0:
            strain = mod_noisy_rms_strain[0]
            max_strain = strain
        else:
            # Grab progressively longer windows and save the biggest strain
            strain = mod_noisy_rms_strain[:k+1] # Has to be k+1 because slicing doesn't include last one
            max_strain = max(strain)
        # Put peak strain back into the output stream
        noisy_peak_strain[k] = max_strain 
    
    mod_noisy_filter_rms_strain = noisy_filter_rms_strain.copy()
    noisy_filter_p_arr_strain = noisy_filter_rms_strain[strain_p_i]
    mod_noisy_filter_rms_strain[:strain_p_i] = noisy_filter_p_arr_strain # Strain before the P wave is set to be the same as the strain AT the P wave
    noisy_filter_peak_strain = mod_noisy_filter_rms_strain.copy()
    for k in range(0,len(mod_noisy_filter_rms_strain)):
        if k == 0:
            strain = mod_noisy_filter_rms_strain[0]
            max_strain = strain
        else:
            # Grab progressively longer windows and save the biggest strain
            strain = mod_noisy_filter_rms_strain[:k+1] # Has to be k+1 because slicing doesn't include last one
            max_strain = max(strain)
        # Put peak strain back into the output stream
        noisy_filter_peak_strain[k] = max_strain 
        
#     # Plot
#     plt.plot(time_strain, rms_strain, label = 'RMS')
#     plt.plot(time_strain, noisy_rms_strain, label = 'Noisy RMS')
#     plt.plot(time_strain, noisy_peak_strain, label = 'Noisy peak')
#     plt.semilogy()
#     plt.legend()
#     plt.xlim(40,60)
#     plt.ylim(10**-12, 10**0)
#     plt.show()
    
#     plt.plot(time_strain, filter_rms_strain, label = 'Filtered RMS')
#     plt.plot(time_strain, noisy_filter_rms_strain, label = 'Filtered noisy RMS')
#     plt.plot(time_strain, noisy_filter_peak_strain, label = 'Filtered noisy peak')
#     plt.semilogy()
#     plt.legend()
#     plt.xlim(40,60)
#     plt.ylim(10**-12, 10**0)
#     plt.show()

    ### Save the new data ###
    
    both_noisy_rms_strain = np.column_stack((time_strain, noisy_rms_strain))
    np.save(sw4_path + 'final_RC_model_outputs_processed/strain/noise_added/rms/' + str(sta) + '.npy', both_noisy_rms_strain)
    
    both_noisy_peak_strain = np.column_stack((time_strain, noisy_peak_strain))
    np.save(sw4_path + 'final_RC_model_outputs_processed/strain/noise_added/peak/' + str(sta) + '.npy', both_noisy_peak_strain)
    
    both_noisy_filter_rms_strain = np.column_stack((time_strain, noisy_filter_rms_strain))
    np.save(sw4_path + 'final_RC_model_outputs_processed/strain/noise_added/filtered/rms/' + str(sta) + '.npy', both_noisy_filter_rms_strain)
    
    both_noisy_filter_peak_strain = np.column_stack((time_strain, noisy_filter_peak_strain))
    np.save(sw4_path + 'final_RC_model_outputs_processed/strain/noise_added/filtered/peak/' + str(sta) + '.npy', both_noisy_filter_peak_strain)

In [ ]:

    
    ### Process GNSS data to match the strain processing ###
    
    # Calculate horizontal RMS GNSS  
    time_gnss = sw4_gnss.time.values
    x_gnss = sw4_gnss.x.values
    y_gnss = sw4_gnss.y.values
    rms_gnss = np.sqrt((x_gnss**2 + y_gnss**2)/2)
    
    # Calculate peak GNSS
    gnss_p_i = np.abs(time_gnss - theo_p_arr).argmin() # Get the index where the P-wave arrives
    mod_rms_gnss = rms_gnss.copy()
    p_arr_gnss = rms_gnss[gnss_p_i]
    mod_rms_gnss[:gnss_p_i] = p_arr_gnss # Strain before the P wave is set to be the same as the strain AT the P wave
    peak_gnss = mod_rms_gnss.copy()
    for k in range(0,len(mod_rms_gnss)):
        if k == 0:
            gnss = mod_rms_gnss[0]
            max_gnss = gnss
        else:
            # Grab progressively longer windows and save the biggest strain
            gnss = mod_rms_gnss[:k+1] # Has to be k+1 because slicing doesn't include last one
            max_gnss = max(gnss)
        # Put peak strain back into the output stream
        peak_gnss[k] = max_gnss 
    
    ### Make plots ###
    
    make_plots(sta, hypo_dist, theo_p_arr, time_strain, rms_strain, peak_strain, time_gnss, rms_gnss, peak_gnss, show_or_save = 'save')
    
    ### Save the new data ###
    
    both_rms_strain = np.column_stack((time_strain, rms_strain))
    np.save(path + 'rms_and_peak_sw4_data/strain/rms/' + str(sta) + '.npy', both_rms_strain)
    
    both_rms_gnss = np.column_stack((time_gnss, rms_gnss))
    np.save(path + 'rms_and_peak_sw4_data/gnss_at_strain_locs/rms/' + str(sta) + '.npy', both_rms_gnss)
    
    both_peak_strain = np.column_stack((time_strain, peak_strain))
    np.save(path + 'rms_and_peak_sw4_data/strain/peak/' + str(sta) + '.npy', both_peak_strain)
    
    both_peak_gnss = np.column_stack((time_gnss, peak_gnss))
    np.save(path + 'rms_and_peak_sw4_data/gnss_at_strain_locs/peak/' + str(sta) + '.npy', both_peak_gnss)
    